# Crawl headlines from nytimes webpage


 This programme can download data(i.e. news title) for the whole specific year from the nytimes news website.

 Change the code where says **# CHANGE HERE**

In [1]:
import os
import pandas as pd
import requests
import json
import time
import dateutil
import datetime
import configparser
from dateutil.relativedelta import relativedelta
import urllib3
from nytimesarticle import articleAPI
from pprint import pprint
from bs4 import BeautifulSoup  
import re
from tqdm import tqdm
import gc
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
def date_format(start, end):
    dates = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime('%Y %m').tolist()]
#     print(dates)
    months_in_range = []
    for date in dates:
        #print(date[1][0])
        if date[1][0]=='0':
            months_in_range.append([date[0],date[1].replace('0','')])
        else:
            months_in_range.append([date[0],date[1]])
    return months_in_range

In [3]:
gc.disable();
gc.enable();
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
YOUR_API_KEY= 'VCdDP0EMPdz3AUYHAkCQ12h6ELuTzJO8'

# CHANGE HERE TO GET DATA OF THE YEAR YOU WANT
end = datetime.date(2013, 1, 1) 
start = datetime.date(2011, 12, 31)
print('Start date: ' + str(start))
print('End date: ' + str(end))
months_in_range=date_format(start,end)
months_in_range

Start date: 2011-12-31
End date: 2013-01-01


[['2012', '1'],
 ['2012', '2'],
 ['2012', '3'],
 ['2012', '4'],
 ['2012', '5'],
 ['2012', '6'],
 ['2012', '7'],
 ['2012', '8'],
 ['2012', '9'],
 ['2012', '10'],
 ['2012', '11'],
 ['2012', '12'],
 ['2013', '1']]

In [4]:
def send_request(date):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + YOUR_API_KEY
    try:
        response = requests.get(url, verify=False).json()
    except Exception:
        return None
    time.sleep(6)
    return response


def is_valid(article, date):
    '''An article is only worth checking if it is in range, and has a headline.'''
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline


def parse_response(response):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
#         'doc_type': [],
#         'material_type': [],
#         'section': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    urls = []
    dates_url = []
#     title = []
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            
            data['headline'].append(article['headline']['main']) 
#             date = article['pub_date'].strftime("%Y%m%d")
#             dat = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %m").tolist()]
            time_stamp = pd.Timestamp(article['pub_date'])
            dates_url.append(time_stamp.strftime("%Y%m%d-%H%M%S"))
            data['date'].append(time_stamp.strftime("%Y%m%d-%H%M%S"))
#             if 'section' in article:
#                 data['section'].append(article['section_name'])
#             else:
#                 data['section'].append(None)
#             data['doc_type'].append(article['document_type'])
#             if 'type_of_material' in article: 
#                 data['material_type'].append(article['type_of_material'])
#             else:
#                 data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
            urls.append(article["web_url"])
    print("Parse articles : " , len(urls))
    return urls,dates_url,data




In [5]:
def get_data(dates):
    '''Sends and parses request/response to/from NYT Archive API for given dates.'''
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    data_list = []
    for date in dates:
        print('Working on ' + str(date) + '...')
#         csv_path = 'headlines/' + date[0] + '-' + date[1] + '.csv'
        txt_path = "nytimes_data/"+ date[0] + '_' + date[1]
#         if not os.path.exists(txt_path): # If we don't already have this month 
        response = send_request(date)
#         if response is not None:
        urls, date_all,data = parse_response(response)
        data_list.append(data)
#         print(data)
        total += len(urls)
#         counter = 0
#         directory = "nytimes_data/"+date[0]+"/"+date[1]
#         if not os.path.exists(directory):
#             os.makedirs(directory)
#         data = []
#         for every_url in tqdm(urls):
            
# #             file = open(directory+"/"+ date_all[counter] + "_"+str(counter)+".txt","w",encoding='utf-8') 
#             data.append(get_info(every_url))
# #             file.write(data)
# #             file.write("\n")
# #             file.close()
#             counter+=1
#         print('Saving ' + txt_path + '...')
# #         pprint(urls)
#         data_list.append(data)
                
# #                 df.to_csv(csv_path, index=False)
#     print(len(data_list))
#     print('Number of articles collected: ' + str(total))
    return urls,data_list

In [77]:
url,data = get_data(months_in_range)

Date range: ['2012', '1'] to ['2013', '1']
Working on ['2012', '1']...
Parse articles :  9168
Working on ['2012', '2']...
Parse articles :  8796
Working on ['2012', '3']...
Parse articles :  9280
Working on ['2012', '4']...
Parse articles :  8240
Working on ['2012', '5']...
Parse articles :  8862
Working on ['2012', '6']...
Parse articles :  8704
Working on ['2012', '7']...
Parse articles :  7795
Working on ['2012', '8']...
Parse articles :  8202
Working on ['2012', '9']...
Parse articles :  8306
Working on ['2012', '10']...
Parse articles :  8731
Working on ['2012', '11']...
Parse articles :  7959
Working on ['2012', '12']...
Parse articles :  7212
Working on ['2013', '1']...
Parse articles :  0


In [78]:
df_data = pd.DataFrame()
for dat in data:
    df_data = df_data.append(pd.DataFrame(dat),ignore_index=True)
df_data

,headline,date,keywords
0,A Year Full of Missteps Is Finally Over,20120101-000028,[Athletics and Sports]
1,Serviceman Held for Carrying Explosives Into T...,20120101-000912,"[Airport Security, AIRPORTS, Bombs and Explosi..."
2,Picturing How Ryan Brothers Could Settle Fami...,20120101-001459,[Football]
3,Experts Weigh In on Outside Groups’ Pro-Gingri...,20120101-001552,"[Campaign Finance, Citizens United v Federal E..."
4,An Unlikely Group Rebels Against Preservation ...,20120101-001554,"[Historic Buildings and Sites, Restoration and..."
...,...,...,...
101250,How Many Slaves Work for You?,20121231-234532,"[Human Trafficking, Emancipation Proclamation ..."
101251,"After Crispy Pig Ears, 10 Trends for 2013",20121231-234720,"[Cooking and Cookbooks, Restaurants, Food]"
101252,Hurting Knicks Uncertain of Tuesday’s Lineup,20121231-234851,[Basketball]
101253,Pursuing a First Win in the Bowls Since 1949,20121231-235117,"[Football (College), Bowl Games]"


**we save data by year into .csv file**

Don't forget to change the path when saving data.

In [79]:
# CHANGE HERE TO SAVE DATA IN YOUR LOCAL STORRAGE
df_data.to_csv(path_or_buf=r'C:\Users\Dell\Desktop\data_with_title\2012_title.csv')
print('data saved')

data saved
